In [ ]:
## Import all the necessary files modules and routines
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
import pandas as pd
import pickle as pkl
tensorflow.keras.backend.clear_session()
from tensorflow.keras.layers import Concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers import Input, add
from keras.models import Model
from matplotlib import image
from matplotlib import pyplot
from ipywidgets import interact
import matplotlib.pyplot as plt
from skimage.transform import resize
from numpy import genfromtxt
from tensorflow.keras.utils import to_categorical

In [ ]:

## Import Low Resolution images as np array and split in (32*32) format 
Y=np.load('../input/dataset-new/Lowres.npy',allow_pickle=True)

x_original=np.split(Y,945)

# Plotting images for easy visualization of Structures
def plot_bridge(i):
    fig,ax = pyplot.subplots()
    im = pyplot.imshow(x_original[i], cmap='gray_r')
    pyplot.axis('off')
    
    pyplot.show()

interact(plot_bridge, i=(0, len(x_original)-1))


In [ ]:
X_input=np.array(x_original)

In [ ]:
X=np.array(x_original)
print(X.shape)
a=np.zeros((1,33))
X= np.insert(X, 0, 1, axis=2)
print(X[944])
print(X.shape)

In [ ]:
# Creating zero valued multi- dimensional array of dimension (945*33*33) to accomdate Boundary conditions.
X_data=np.zeros((945,33,33))
print(a.shape)
for i in range(945):
    X_data[i]=np.append(X[i],a,axis=0)
print(X_data.shape)

In [ ]:
# Creating a channel for consideration of X-directional fixed BC
X_data3=np.expand_dims(X_data,axis=3)
print(X_data3.shape)

In [ ]:
# Creating a channel for consideration of Y-directional fixed BC
X_data4=X_data3
print(X_data4.shape)

In [ ]:
#Creating a channel for consideration of X-directional force BC
X_data1_temp= np.insert(x_original, 0, 0, axis=2)
X_data1=np.zeros((945,33,33))
for i in range(945):
    X_data1[i]=np.append(X_data1_temp[i],a,axis=0)

In [ ]:
#Creating a channel for consideration of Y-directional force BC
X_data2 = X_data1
for i in range(945):
    X_data2[i][32][32]=1

X_data2=np.expand_dims(X_data2,axis=3)
print(X_data2.shape)

In [ ]:
X_data1=np.expand_dims(X_data1,axis=3)
print(X_data1.shape)

In [ ]:
#Concatenating all the individual channels (BCs) into one, creating final input 

X_final= np.concatenate((X_data1, X_data2,X_data3,X_data4),axis=3)
print(X_final.shape)

In [ ]:
#Importing Volume fraction 

my_data = genfromtxt('../input/dataset-new/input.csv', delimiter=',')[:,2]
my_data = my_data[0:945]
print(my_data[30:40])

**CNN AUTOENCODER**

In [ ]:
# Creating volume fraction data

X_data_parent=np.ones((105,4,4,1))
a1 = X_data_parent*0.1
a2= X_data_parent*0.2
a3= X_data_parent*0.3
a4= X_data_parent*0.4
a5= X_data_parent*0.5
a6= X_data_parent*0.6
a7= X_data_parent*0.7
a8= X_data_parent*0.8
a9= X_data_parent*0.9
abc=np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9),axis=0)

print(abc.shape)

In [ ]:
#Creating encoder-decoder model

start = Input(shape=(33, 33 ,4)) 

#ENCODER
conv1_1= Conv2D(64,(2,2),activation ='relu',strides=(1,1))(start)
conv1_2= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv1_1)
pool1= MaxPooling2D(pool_size=(2,2))(conv1_2)
conv1_3= Conv2D(128,(3,3),padding = "same",activation ='relu',strides=(1,1))(pool1)
conv1_4= Conv2D(128,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv1_3)
pool2= MaxPooling2D(pool_size=(2,2))(conv1_4)
conv1_5= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(pool2)
conv1_6= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv1_5)
pool3= MaxPooling2D(pool_size=(2,2))(conv1_6)
h= Conv2D(512,(3,3),strides=(1, 1),padding = 'same',activation ='relu')(pool3)

label=Input(shape=(4,4,1))

lat=Concatenate()([h,label])

#DECODER

conv2_1= Conv2D(512,(3,3),padding="same",activation ='relu',strides=(1,1))(lat)
up1= UpSampling2D(size=(2,2))(conv2_1)
conv2_2= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(up1)
conv2_3= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv2_2)
up2= UpSampling2D(size=(2,2))(conv2_3)
conv2_4= Conv2D(128,(3,3),padding = "same",activation ='relu',strides=(1,1))(up2)
conv2_5= Conv2D(128,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv2_4)
up3= UpSampling2D(size=(2,2))(conv2_5)
conv2_6= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(up3)
conv2_7= Conv2D(2,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv2_6)
r= Conv2D(1,(3,3),padding = "same",activation ='sigmoid',strides=(1,1))(conv2_7)



In [ ]:
#Assigning inputs and outputs to the autoencoder model and compiling the same

autoencoder = Model(inputs=[start,label], outputs=r)
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()

In [ ]:
# test train split of training and validation data

images_train, images_test,abc_train,abc_test = train_test_split(X_final,abc,train_size= 0.8,test_size=0.20,shuffle=False)
print(images_train.shape)
print(abc_train.shape)

X_input_train = X_input[:756]
print(X_input_train.shape)
X_input_test = X_input[756:]
print(X_input_test.shape)


In [ ]:
# Assigning Hyperparameters (batch size and epochs) and training the model

epochs = 12
batch_size = 50

history = autoencoder.fit([images_train,abc_train], X_input_train , batch_size=batch_size, epochs=epochs,verbose=1, validation_data=([images_test,abc_test],X_input_test))

###### Predicting on test data

decoded_imgs = autoencoder.predict([images_test, abc_test])



In [ ]:
decoded_imgs = autoencoder.predict([images_test, abc_test])

In [ ]:
#Plotting interactive images of predicted images on test_data


def plot_bridge(i):
    fig,ax = pyplot.subplots()
    
    im = pyplot.imshow(decoded_imgs[i].reshape(32,32), cmap='gray_r')
    pyplot.axis('off')
    
    pyplot.show()

interact(plot_bridge, i=(0, len(decoded_imgs)-1))

In [ ]:
#Plotting interactive images of original images(ground truth) on test_data

def plot_bridge1(i):
    fig,ax = pyplot.subplots()
    
    #print(f"Volume fraction: {y[i]}")
    im1 = pyplot.imshow(X_input_test[i].reshape(32,32), cmap='gray_r')
    pyplot.axis('off')
    
    pyplot.show()
 
interact(plot_bridge1, i=(0, len(X_input_test)-1))

In [ ]:
#Plotting training and validation losses against epoch

print(history.history.keys())

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## GAN 

Generator module


In [ ]:
#Importing all the necessary modules and routines for GAN

import tensorflow as tf
from tensorflow import keras
from PIL import Image
from PIL.ImageColor import getrgb
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, Flatten
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import shutil
import datetime
%load_ext tensorboard

In [ ]:
#Predicting on full data set as it will be input to GAN

input_GAN = autoencoder.predict([X_final,abc])

def plot_bridge3(i):
    fig,ax = pyplot.subplots()
    
    im3 = pyplot.imshow(input_GAN[i].reshape(32,32), cmap='gray_r')
    pyplot.axis('off')
    
    pyplot.show()
 
 
interact(plot_bridge3, i=(0, len(input_GAN)-1))

In [ ]:
#Building generator network

def build_generator():
   
    residual_blocks = 16
    input_shape = (32, 32, 1)

    # Input Layer of the generator network
    input_layer = Input(shape=input_shape)

    # Add the pre-residual block
    conv1_GeN = Conv2D(64,(5,5),activation ='relu',padding='same',strides=(1,1))(input_layer)

    # Add the post-residual block

    conv2_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv1_GeN)
    conv3_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv2_GeN)
    conv4_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv3_GeN)
    conv5_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv4_GeN)
    conv6_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv5_GeN)
    conv7_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv6_GeN)
    conv8_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv7_GeN)
    conv9_GeN= Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv8_GeN)

    conv10_GeN = Conv2D(64,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv9_GeN)
    Residual_inter_output= BatchNormalization(momentum = 0.8)(conv10_GeN)

    # Take the sum of the output from the pre-residual block(gen1) and the post-residual block(gen2)
    Residual_output = Add()([Residual_inter_output, conv1_GeN])

    # Add an upsampling block
    conv11_GeN = UpSampling2D(size=(2,2))(Residual_output) 
    conv12_GeN= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv11_GeN)
    
    
    # Add another upsampling block
    conv13_GeN = UpSampling2D(size=(2,2))(conv12_GeN)
    conv14_GeN= Conv2D(256,(3,3),padding = "same",activation ='relu',strides=(1,1))(conv13_GeN)

    # Output convolution layer
    conv15_GeN= Conv2D(1,(5,5),padding = "same",activation ='sigmoid',strides=(1,1))(conv14_GeN)

    # Keras model
    model = Model(inputs=[input_layer], outputs=[conv15_GeN], name='generator')
    return model

In [ ]:
#Building discriminator network

def build_discriminator():
  
    leakyrelu_alpha = 0.2
    input_shape = (128,128,2)

    input_layer = Input(shape=input_shape)

    # Add the first convolution block
    dis1 = Conv2D(64,(3,3), strides=1, padding='same')(input_layer)
    dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)

    # Add the 2nd convolution block
    dis2 = Conv2D(64,(3,3), strides=2, padding='same')(dis1)
    dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
    dis2 = BatchNormalization(momentum = 0.8)(dis2)

    # Add the third convolution block
    dis3 = Conv2D(128,(3,3), strides=1,padding = 'same')(dis2)
    dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
    dis3 = BatchNormalization(momentum = 0.8)(dis3)

    # Add the fourth convolution block
    dis4 = Conv2D(128,(3,3), strides=2, padding = 'same')(dis3)
    dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
    dis4 = BatchNormalization(momentum = 0.8)(dis4)

    # Add the fifth convolution block
    dis5 = Conv2D(256,(3,3), strides=1,padding='same')(dis4)
    dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
    dis5 = BatchNormalization(momentum = 0.8)(dis5)

    # Add the sixth convolution block
    dis6 = Conv2D(256,(3,3), strides=2, padding = 'same')(dis5)
    dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
    dis6 = BatchNormalization(momentum = 0.8)(dis6)

    # Add the seventh convolution block
    dis7 = Conv2D(512,(3,3), strides=1,padding='same')(dis6)
    dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
    dis7 = BatchNormalization(momentum = 0.8)(dis7)

    # Add the eight convolution block
    dis8 = Conv2D(512,(3,3), strides=2, padding = 'same')(dis7)
    dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
    dis8 = BatchNormalization(momentum = 0.8)(dis8)

    # Add a dense layer
    flat = keras.layers.Flatten()(dis8)
    dis9 = Dense(units=1024)(flat)
    dis9 = LeakyReLU(alpha=0.2)(dis9)

    # Last dense layer - for classification
    output = Dense(units=1, activation='sigmoid')(dis9)

    model = Model(inputs=[input_layer], outputs=[output], name='discriminator')
    return model

In [ ]:
# Importing Pretrained weights of Imagenet data set in VGG19 architecture at 13th layer

def build_vgg():
    """Build VGG network to extract image features """
    grayImage = (128,128,3)

    vgg = keras.applications.VGG19(include_top = False ,  input_shape = grayImage , weights="imagenet")
    features = vgg.get_layer(index = 13).output
    #features = vgg.output
    model = keras.Model(inputs=[vgg.inputs], outputs=[features])
    return model

        




### Model

In [ ]:
# Building Adverserial (combination of generator and discriminator) network

mode = 'train'

epochs = 30
batch_size = 20

common_optimizer = Adam(0.0002, 0.5)
low_resolution_shape = (32, 32, 1)
high_resolution_shape = (128, 128,1)

vgg = build_vgg()
vgg.trainable = False
vgg.compile(loss='mae', optimizer=common_optimizer, metrics=['accuracy'])

# Build and compile the discriminator network
discriminator = build_discriminator()
discriminator.compile(loss='mae', optimizer=common_optimizer, metrics=['accuracy'])

# Build the generator network
generator = build_generator()

"""
Build and compile the adversarial model
"""

# Input layers for high-resolution and low-resolution images
input_high_resolution = Input(shape=high_resolution_shape)
input_low_resolution = Input(shape=low_resolution_shape)
#Input_final_to_discriminator=Input(shape=Input_to_discriminator)

# Generate high-resolution images from low-resolution images
generated_high_resolution_images = generator(input_low_resolution)

rgbImage=Concatenate()([generated_high_resolution_images,generated_high_resolution_images,generated_high_resolution_images])

# Extract feature maps of the generated images
features = vgg(rgbImage)

# Freeze the discriminator for while training the adversarial model
discriminator.trainable = False

final_input_to_discriminator=Concatenate()([input_high_resolution,input_high_resolution])
# Get the probability of generated high-resolution images
probs = discriminator(final_input_to_discriminator)

# Create and compile an adversarial model
adversarial_model = Model([input_low_resolution, input_high_resolution], [probs,features])
adversarial_model.compile(loss=['binary_crossentropy', 'mae'], loss_weights=[1e-3, 1], optimizer=common_optimizer)

generator.summary()
discriminator.summary()
adversarial_model.summary()

In [ ]:
#Importing high reolution images(ground truth) and splitting data for training and testing

low_resolution_images = input_GAN
inverted_high_resolution_img =np.load('../input/dataset-new/High_res.npy',allow_pickle=True)

low_resolution_images_train,low_resolution_images_test,inverted_high_resolution_img_train,inverted_high_resolution_img_test = train_test_split(low_resolution_images,inverted_high_resolution_img,train_size= 0.8,test_size=0.20,shuffle=False)
print(low_resolution_images_train.shape)
print(inverted_high_resolution_img_test.shape)
print(low_resolution_images.shape)

In [ ]:
#temporary modification of ground truth data to be passed in VGG19 network


inverted_high_resolution_img1=np.expand_dims(inverted_high_resolution_img_train,axis=3)
inverted_high_resolution_img2=np.concatenate((inverted_high_resolution_img1,inverted_high_resolution_img1,inverted_high_resolution_img1),axis=3)


#UPSCALING OF 32x32: 
resized_low_resolution_image = resize(low_resolution_images, (945,128,128,1),order =0,preserve_range=True, anti_aliasing=False)
print(resized_low_resolution_image.shape)
print(inverted_high_resolution_img.shape)
expanded = np.expand_dims(inverted_high_resolution_img,axis=3)
discrim_realdata= np.concatenate((resized_low_resolution_image,expanded),axis=3)
print(discrim_realdata.shape)
discrim_realdata_train=discrim_realdata[:756]
print(discrim_realdata_train.shape)
discrim_realdata_test=discrim_realdata[756:]
print(discrim_realdata_test.shape)

#SPLIT THE RESIZED DATA TO TRAIN AND TEST
resized_low_resolution_image_train = resized_low_resolution_image[:756]
resized_low_resolution_image_test = resized_low_resolution_image[756:]
print(resized_low_resolution_image_train.shape)
print(resized_low_resolution_image_test.shape)


In [ ]:
# Training Adverserial Model
def just_train():
    
    train_batches = int(np.ceil(len(low_resolution_images_train)/batch_size))-1
    
    for epoch in range(epochs):
        
        for i in  range(train_batches):
   
            beg = i*batch_size
            end = (i+1)*batch_size

            # Generate high-resolution images from low-resolution images
            generated_high_resolution_images = generator.predict(low_resolution_images_train[beg:end])
            discrim_fakedata = np.concatenate((resized_low_resolution_image_train[beg:end],generated_high_resolution_images),axis=3)
                          
            # Generate batch of real and fake labels
            real_labels = np.ones((batch_size, 1))*0.95
            fake_labels = np.ones((batch_size,  1))*0.05
                          
            # Train the discriminator network on real and fake images
            d_loss_real = discriminator.train_on_batch(discrim_realdata[beg:end], real_labels)
            d_loss_fake = discriminator.train_on_batch(discrim_fakedata, fake_labels)

            # Calculate total discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            image_features = vgg.predict(inverted_high_resolution_img2[beg:end])

            # Train the generator network
            g_loss = adversarial_model.train_on_batch([low_resolution_images_train[beg:end],inverted_high_resolution_img_train[beg:end]], [real_labels,image_features])


            print("Epoch {} : g_loss: {} , d_loss: {}".format(epoch , g_loss[0] , d_loss[0]))

    
if mode == 'train':
        just_train()


In [ ]:
# Predict on test data and plotting interactively

generated_images = generator.predict(low_resolution_images_test)

def plot_bridge4(i):
    fig,ax = pyplot.subplots()
    im4 = pyplot.imshow(generated_images[i].reshape(128,128), cmap='gray')
    pyplot.axis('off')
    
    pyplot.show()

interact(plot_bridge4, i=(0, len(generated_images)-1))